<a href="https://colab.research.google.com/github/Phurin216/bsc_dpdm24/blob/main/%E0%B8%99%E0%B8%B2%E0%B8%A2%E0%B8%A0%E0%B8%B9%E0%B8%A3%E0%B8%B4%E0%B8%99%E0%B8%97%E0%B9%8C_%E0%B8%AA%E0%B8%B2%E0%B8%A2%E0%B9%80%E0%B8%8A%E0%B8%B7%E0%B9%89%E0%B8%AD_653020216_6_midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [2]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/MyDrive/Midterm')

#เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก google drive โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone' จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

In [5]:
Datahouse = pd.read_csv('/content/drive/MyDrive/Midterm/Data House - บ้านมือ2.csv')

In [6]:
Datahouse

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,NaN,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,NaN,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,NaN,NaN,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,NaN,NaN,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,NaN,NaN,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,NaN,NaN,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,NaN,NaN,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,ไม่ระบุ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
Datahouse.columns

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden'],
      dtype='object')

In [8]:
import pandas as pd

# โหลดไฟล์ CSV
file_path = '/path/to/your/Data House.csv'  # ใส่เส้นทางไฟล์ CSV ที่คุณแปลงแล้ว
data = pd.read_csv('/content/drive/MyDrive/Midterm/Data House - บ้านมือ2.csv')

# เลือกเฉพาะคอลัมน์ที่ต้องการ
columns = [
    'renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
    'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
    'direction', 'furniture', 'usable_area', 'number_of_parking',
    'sell_price', 'district', 'sub_district', 'zone'
]
data_filtered = data[columns]

# ลบแถวที่มีค่า NaN (ถ้ามี)
data_filtered = data_filtered.dropna()

# แสดงข้อมูลที่เตรียม
print(data_filtered.head())

# บันทึกข้อมูลที่เตรียมเป็น CSV ใหม่ (ถ้าต้องการ)
data_filtered.to_csv('prepared_data.csv', index=False)


  renovate  number_of_rai  number_of_ngan  number_of_va  number_of_story  \
3       no              0               0          61.6              2.0   
4       no              0               0          72.3              2.0   
5       no              0               0          65.8              2.0   
6       no              0               1           0.0              2.0   
7       no              0               0          63.8              1.0   

   number_of_bedroom  number_of_bathroom direction furniture  usable_area  \
3                3.0                   3   ไม่ระบุ   ไม่ระบุ        203.0   
4                3.0                   2   ไม่ระบุ   ไม่ระบุ        164.0   
5                3.0                   3   ไม่ระบุ   ไม่ระบุ        185.0   
6                7.0                   3   ไม่ระบุ   ไม่ระบุ        383.0   
7                3.0                   2   ไม่ระบุ   ไม่ระบุ        135.0   

   number_of_parking  sell_price      district sub_district  \
3                

#จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school') จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ 7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

In [9]:
# prompt: จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school') จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ 7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

import pandas as pd
import json
from geopy.distance import geodesic

# Load data
house_data = pd.read_csv('/content/drive/MyDrive/Midterm/Data House - บ้านมือ2.csv')
with open('/content/drive/MyDrive/Midterm/7-eleven.json', 'r') as f:
    seven_eleven_data = json.load(f)
with open('/content/drive/MyDrive/Midterm/school.json', 'r') as f:
    school_data = json.load(f)

In [49]:
# Initialize new columns
house_data['d1000-7/11'] = 0
house_data['d1000-school'] = 0

# Function to count nearby locations
def count_nearby(house_lat, house_lon, locations, radius_km):
    count = 0
    for location in locations:
        try:
            location_lat = location['latitude']
            location_lon = location['longitude']
            distance = geodesic((house_lat, house_lon), (location_lat, location_lon)).km
            if distance <= radius_km:
                count += 1
        except:
            pass  # Handle potential errors in location data
    return count

# Iterate through houses and count nearby locations
for index, row in house_data.iterrows():
    try:
      house_lat = row['latitude']
      house_lon = row['longitude']
      house_data.loc[index, 'd1000-7/11'] = count_nearby(house_lat, house_lon, seven_eleven_data, 1)
      house_data.loc[index, 'd1000-school'] = count_nearby(house_lat, house_lon, school_data, 1)
    except:
        pass # Handle cases where latitude/longitude is missing

# Print the updated dataframe
print(house_data[['id', 'd1000-7/11', 'd1000-school']].head())
print('ID บ้านหลังดังกล่าว ไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร')

   id  d1000-7/11  d1000-school
0   1           0             0
1   4           0             0
2  16           0             0
3  18           0             0
4  19           0             0
ID บ้านหลังดังกล่าว ไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร


#ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

In [82]:
# prompt: ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID) - บ้าน.csv" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# Load the appraiser data
appraiser_data = pd.read_csv('/content/drive/MyDrive/Midterm/คำตอบนักประเมิน(ID) - บ้าน.csv')

appraiser_data = appraiser_data.rename(columns={
    'ประเมินราคาซื้อ-ขาย': 'estimator',
    'ให้คะแนนทรัพย์': 'score'
})
# Filter for appraiser ID 01
appraiser_01_data = appraiser_data[appraiser_data['ID'] == '01']

In [85]:
appraiser_data

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,estimator,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,500,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,"2,890,000","2,700,000","โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",NaN,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,200,000","5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,"2,600,000","2,400,000","ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,"6,050,000","5,000,000","5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1912,2,https://nayoo.co/khonkaen/posts/17867,17867,บ้านเดี่ยว,มั่นใจ,"965,000","550,000","ราคาเสนอขาย 2,000 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1913,2,https://nayoo.co/khonkaen/posts/17873,17873,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,000,000","ราคาเสนอขาย 2,500 - 3,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1914,2,https://nayoo.co/khonkaen/posts/17874,17874,บ้านเดี่ยว,มั่นใจ,"10,873,000","10,000,000","ราคาเสนอขาย 12,000 - 15,000 บาท/ตารางวา",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1915,2,https://nayoo.co/khonkaen/posts/17876,17876,บ้านเดี่ยว,มั่นใจ,"1,000,000","1,200,000",ราคาเสนอขาย 1.2-1.5 ล้านบาท/หลัง,NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [86]:
print(merged_data['score'].isnull().sum())  # ตรวจสอบจำนวนค่า NaN ในคอลัมน์


0


In [87]:
merged_data['score'] = merged_data['score'].fillna(0)  # เติมค่า 0 ให้กับ NaN


In [88]:
subdistrict_with_highest_score = mean_score_by_subdistrict.sort_values(ascending=False).index[0]


#1.zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

In [38]:
# prompt: zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

# Group data by 'zone' and calculate the mean 'sell_price' for each zone
mean_sell_price_by_zone = data_filtered.groupby('zone')['sell_price'].mean()

# Find the zone with the highest mean 'sell_price'
zone_with_highest_price = mean_sell_price_by_zone.idxmax()

# Print the result
print(f"โซนที่มีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด คือ {zone_with_highest_price}")

โซนที่มีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด คือ ในเมือง-รื่นรมย์-เทพารักษ์


#2.บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)

In [39]:
# prompt: บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)

# Find the house with the maximum number of 7-11s within a 1km radius
max_711_count = house_data['d1000-7/11'].max()
house_with_max_711 = house_data[house_data['d1000-7/11'] == max_711_count]

# Get the price of that house
price_of_house_with_max_711 = house_with_max_711['sell_price'].iloc[0]

# Print the result
print(f"บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคา {price_of_house_with_max_711}")

บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคา 390000.0


#3.ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [70]:
# เติมค่า NaN ในคอลัมน์ 'score' และแปลงเป็น float
merged_data['score'] = merged_data['score'].fillna(0).astype(float)

# คำนวณคะแนนเฉลี่ยต่อ sub_district
mean_score_by_subdistrict = merged_data.groupby('sub_district')['score'].mean()

# ตรวจสอบว่ามีค่าใน mean_score_by_subdistrict หรือไม่
if not mean_score_by_subdistrict.empty:
    subdistrict_with_highest_score = mean_score_by_subdistrict.sort_values(ascending=False).index[0]
    print(f"ตำบลที่มีคะแนนทรัพย์ 'score' เฉลี่ยจากการประเมินมากที่สุดคือ: {subdistrict_with_highest_score}")
else:
    print("ไม่มีข้อมูลคะแนนทรัพย์ใน mean_score_by_subdistrict")


ตำบลที่มีคะแนนทรัพย์ 'score' เฉลี่ยจากการประเมินมากที่สุดคือ: กุดกว้าง
